In [21]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 4
atoms = ""
for i in range(nH):
    atoms += f"O {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit='bohr', spin=0, verbose=4)
mol.build()

mf = scf.RHF(mol)
mf.kernel()

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sat Nov  8 22:59:00 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 32
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 O      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00000

SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /tmp/tmpo9y7tuxj
max_memory 4000 MB (current use 429 MB)
Set gradient conv threshold to 3.16228e-05
Initial guess from minao.
init E= -301.396336095261

WARN: HOMO 0.0667032086944778 == LUMO 0.0667032086944786

cycle= 1 E= -296.923271571486  delta_E= 4.47  |g|= 0.71  |ddm|= 4.09
  HOMO = -0.21390243597596  LUMO = 0.162544766357426
cycle= 2 E= -297.047548019924  delta_E= -0.124  |g|= 0.212  |ddm|= 1.32
  HOMO = -0.203689403026691  LUMO = 0.206191778589984
cycle= 3 E= -297.061167070435  delta_E= -0.0136  |g|= 0.0261  |ddm|= 0.316
  HOMO = -0.204355841751843  LUMO = 0.209730992977326
cycle= 4 E= -297.061462532385  delta_E= -0.000295  |g|= 0.00683  |ddm|= 0.058
  HOMO = -0.204508690852695  LUMO = 0.209159482369177
cycle= 5 E= -297.061491134214  delta_E= -2.86e-05  |g|= 0.00113  |ddm|= 0.0202
  HOMO = -0.204645020238827  LUMO = 0.209312657353877
cycle= 6 E=

-0.19679411934115273

In [23]:
mycc.t1 = 10 * mycc.t1

In [24]:
eris = mycc.ao2mo(mycc.mo_coeff)
eccs = mycc.energy(mycc.t1, 0*mycc.t2, eris)
print('mf energy: ',mf.e_tot)
print('ccs energy: ', mf.e_tot+eccs)
# mycc.t1 = (mycc.t1[0]*10,mycc.t1[1]*10)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
print('ccsd energy: ', mf.e_tot+eccsd)

mf energy:  -297.06149197966647
ccs energy:  -297.04842823649113
ccsd energy:  -297.24535299593265


In [25]:
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 3,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'ccsd_pt2',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import config
from ad_afqmc.prop_unrestricted import prop_unrestricted
import time
from jax import random
# from ad_afqmc.ccsd_pt import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
# mycc.t1 = [10*mycc.t1[0],10*mycc.t1[1]]
# ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
prop_unrestricted.prep_afqmc(mycc,options,chol_cut=1e-8)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (prop_unrestricted._prep_afqmc(options))


#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (16, 16)
# Number of basis functions: 20
# Number of Cholesky vectors: 135
#
# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 20
# nelec: (16, 16)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 3
# seed: 2
# walker_type: rhf
# trial: ccsd_pt2
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [42]:
from jax import lax
import opt_einsum as oe
@partial(jit, static_argnums=0)
def _calc_energy_pt(
    trial, walker: jax.Array, ham_data: dict, wave_data: dict
):
    nocc = trial.nelec[0]
    mo_t, t2 = wave_data["mo_t"], wave_data["t2"]
    chol = ham_data["chol"].reshape(-1, trial.norb, trial.norb)
    green = (walker @ (jnp.linalg.inv(mo_t.T @ walker)) @ mo_t.T).T
    greenp = (green - jnp.eye(trial.norb))[:,nocc:]

    h1 = (ham_data["h1"][0] + ham_data["h1"][1]) / 2.0
    hg = oe.contract("pq,pq->", h1, green, backend="jax")
    e1_0 = 2 * hg

    # double excitations
    t2g_c = oe.contract("iajb,ia->jb", t2, green[:nocc,nocc:], backend="jax")
    t2g_e = oe.contract("iajb,ib->ja", t2, green[:nocc,nocc:], backend="jax")
    t2_green_c = (greenp @ t2g_c.T) @ green[:nocc,:]
    t2_green_e = (greenp @ t2g_e.T) @ green[:nocc,:]
    t2_green = 2 * t2_green_c - t2_green_e
    t2g = 2 * t2g_c - t2g_e
    gt2g = oe.contract("ia,ia->", t2g, green[:nocc,nocc:], backend="jax")
    e1_2_1 = 2 * hg * gt2g
    e1_2_2 = -2 * oe.contract("ij,ij->", h1, t2_green, backend="jax")
    e1_2 = e1_2_1 + e1_2_2 # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>

    # two body energy
    # ref
    lg = oe.contract("gpq,pq->g", chol, green, backend="jax")
    # lg1 = oe.contract("gpr,qr->gpq", chol, green, backend="jax")
    # e2_0_1 = 2 * lg @ lg
    # e2_0_2 = -jnp.sum(vmap(lambda x: x * x.T)(lg1))
    # e2_0 = e2_0_1 + e2_0_2

    # double excitations
    # e2_2_1 = e2_0 * gt2g
    lt2g = oe.contract("gij,ij->g", chol, t2_green, backend="jax")
    e2_2_2_1 = -lt2g @ lg

    def scanned_fun(carry, x):
        chol_i = x
        # e2_0
        lg_i = oe.contract("pr,qr->pq", chol_i, green, backend="jax")
        e2_0_1_i = (2*jnp.trace(lg_i))**2 / 2.0
        e2_0_2_i = - oe.contract('pq,qp->',lg_i,lg_i, backend="jax")
        carry[0] += e2_0_1_i + e2_0_2_i
        # e2_2
        gl_i = oe.contract("pr,rq->pq",green,chol_i,backend="jax")
        lt2_green_i = oe.contract("pr,qr->pq",chol_i,t2_green,backend="jax")
        carry[1] += 0.5 * oe.contract("pq,pq->",gl_i,lt2_green_i,backend="jax")
        glgp_i = oe.contract("iq,qa->ia", gl_i[:nocc,:],greenp,backend="jax")
        l2t2_1 = oe.contract("ia,jb,iajb->",glgp_i,glgp_i,t2,backend="jax")
        l2t2_2 = oe.contract("ib,ja,iajb->",glgp_i,glgp_i,t2,backend="jax")
        carry[2] += 2 * l2t2_1 - l2t2_2
        return carry, 0.0

    [e2_0, e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0,0.0,0.0], chol)
    e2_2_1 = e2_0 * gt2g
    e2_2_2 = 4 * (e2_2_2_1 + e2_2_2_2)
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3

    # <HF|walker>
    o0 = jnp.linalg.det(walker[:nocc,:nocc]) ** 2
    # <exp(T1)HF|walker>/<HF|walker>
    t1 = jnp.linalg.det(wave_data["mo_t"].T.conj() @ walker)**2 / o0
    t2 = gt2g * t1 # <exp(T1)HF|T2|walker>/<HF|walker>
    e0 = (e1_0 + e2_0) * t1 # <exp(T1)HF|h1+h2|walker>/<HF|walker>
    e1 = (e1_2 + e2_2) * t1 # <exp(T1)HF|T2 (h1+h2)|walker>/<HF|walker>

    return jnp.real(t1), jnp.real(t2), jnp.real(e0), jnp.real(e1)

In [43]:
config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

prop_data["key"] = random.PRNGKey(seed + rank)
print('init walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
walker = prop_data['walkers']
# et1 = jnp.real(trial._calc_energy(
#     walker_up, walker_dn, ham_data, wave_data
#     ))
# print('exact T1 transformed init walker energy: ', et1)
et1 = trial._calc_energy_restricted(
    wave_data['mo_t'],ham_data,wave_data)
print('exact T1 init walker energy: ', et1)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(mycc.t1, 0*mycc.t2, eris)
print('ccs energy: ', mf.e_tot+eccsd)
print('ccsd energy: ', mycc.e_tot)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init walker energy:  -297.06149193168835
mf enegry:  -297.06149197966647
exact T1 init walker energy:  -297.04842818927625
ccs energy:  -297.04842823649113
ccsd energy:  -297.25828609900765


In [44]:
# mycc.t1 = (mycc.t1[0]*0, mycc.t1[1]*0)
eris = mycc.ao2mo(mycc.mo_coeff)
eccs = mycc.energy(mycc.t1,0*mycc.t2, eris)
eccs = mf.e_tot + eccs
print('ccs energy: ', eccs)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
eccsd = mf.e_tot + eccsd
print('ccsd energy: ', eccsd)

ccs energy:  -297.04842823649113
ccsd energy:  -297.24535299593265


In [48]:
h0 = ham_data['h0']
walker = prop_data['walkers'][0]
import time
start = time.perf_counter()
t1,t2,e0,e1 = _calc_energy_pt(trial, walker, ham_data, wave_data)
end = time.perf_counter()
print(t1,t2,e0,e1)
print(h0 + 1/t1 * e0 )
print(h0 + 1/t1 * e0 - eccs)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0 - eccsd)
print(end-start)

0.7266539021504438 0.0 -316.6140739029216 -0.1430961445463294
-297.04842818927614
4.721499635707005e-08
-297.2453529482609
4.767173322761664e-08
0.0006552999839186668
